In [ ]:
import pims_nd2
from mpl_toolkits.axes_grid1.anchored_artists import AnchoredSizeBar
import matplotlib.pylab as plt
import matplotlib.animation as animation
import subprocess
import labrotation.file_handling as fh
import os
import datadoc_util
import h5py
import scipy
import numpy as np
from scipy import ndimage
import pandas as pd

In [ ]:
i_sz = 0# first sz: 0, second: 1 ...

In [ ]:
env_dict = dict()
if not os.path.exists("./.env"):
    print(".env does not exist")
else:
    with open("./.env", "r") as f:
        for line in f.readlines():
            l = line.rstrip().split("=")
            env_dict[l[0]] = l[1]
print(env_dict.keys())

In [ ]:
if "DATA_DOCU_FOLDER" in env_dict.keys():
    data_docu_folder = env_dict["DATA_DOCU_FOLDER"]
else:
    data_docu_folder = fh.open_dir("Open Data Documentation folder")
print(data_docu_folder)

In [ ]:
ddoc = datadoc_util.DataDocumentation(data_docu_folder)
ddoc.loadDataDoc()

In [ ]:
parameters_log = dict()

In [ ]:
whole_traces_h5_fpath = fh.open_file("Open traces h5 file!")
print(whole_traces_h5_fpath)
parameters_log["filename"] = os.path.split(whole_traces_h5_fpath)[-1]

In [ ]:
grid_fpath = whole_traces_h5_fpath.replace("_traces", "_grid")
print(grid_fpath)

## Read out files

In [ ]:
with h5py.File(whole_traces_h5_fpath, 'r') as hf:
    session_uuid = hf.attrs["uuid"]
    moco_intervals = hf["moco_intervals"][()]
    moco_flags = hf["moco_flags"][()]
    cnmf_intervals = hf["cnmf_intervals"][()]
    cnmf_flags = hf["cnmf_flags"][()]
    begin_end_frames = hf["begin_end_frames"][()]
    # spatial components: CNMF A field
    A_data = hf["spatial"]["data"][()]
    A_indices = hf["spatial"]["indices"][()]
    A_indptr = hf["spatial"]["indptr"][()]
    A_shape = hf["spatial"]["shape"][()]
    # temporal signals, i.e. neuron traces
    temporal = hf["traces"][()]
    mean_fluo = hf["mean_fluo"][()]

In [ ]:
df_onsets = pd.read_hdf(grid_fpath)

In [ ]:
session_files = ddoc.getSessionFilesForUuuid(uuid=session_uuid)
fpath_nd2 = os.path.join(session_files["folder"].iloc[0], session_files["nd2"].iloc[0])
fpath_nd2 = "R" + fpath_nd2[1:]  # currently, server is drive R

In [ ]:
nd2_file = pims_nd2.ND2_Reader(fpath_nd2)
nd2_data = np.array(nd2_file)

## Select data for chosen seizure (i_sz)

In [ ]:
if "i_sz" in df_onsets.keys():
    print("Multiple seizures detected")
    if "df_onsets_all_sz" not in locals():
        df_onsets_all_sz = df_onsets.copy()
    df_onsets = df_onsets_all_sz[df_onsets_all_sz["i_sz"] == i_sz]

In [ ]:
onsets_szs_np = np.array(df_onsets["onset_sz"])
if "onset1" in df_onsets.keys():
    onsets_sd1_np = np.array(df_onsets["onset1"])
if "onset2" in df_onsets.keys():
    onsets_sd2_np = np.array(df_onsets["onset2"])

In [ ]:
n_neurons, n_frames = temporal.shape

In [ ]:
# convert spatial from sparse matrix into dense matrix of proper dimensions
spatial = scipy.sparse.csc.csc_matrix((A_data, A_indices, A_indptr), shape=A_shape).todense()
spatial = np.array(spatial)  # change type to allow np.reshape (?)
spatial = np.reshape(spatial, (512, 512, n_neurons)) # (262144 -> 512x512, i.e. "unflatten")
spatial = np.transpose(spatial, axes=[2, 1,0])  # move neuron index to end

In [ ]:
res_x, res_y = spatial[0].shape

In [ ]:
centers_x = np.zeros((n_neurons))
centers_y = np.zeros((n_neurons))
for i_neuron in range(n_neurons):
    xy = ndimage.measurements.center_of_mass(spatial[i_neuron].T) # transpose needed so that imshow and scatter plotting match 
    centers_x[i_neuron] = xy[0]
    centers_y[i_neuron] = xy[1]

In [ ]:
df_segments = ddoc.getSegmentsForUUID(session_uuid)

In [ ]:
begin_frame_i_sz = df_segments[df_segments["interval_type"] == "sz"].frame_begin.iloc[i_sz]
end_frame_i_sz = 0

for i_row, row in df_segments[df_segments["frame_begin"] >= begin_frame_i_sz].iterrows():
    if row["interval_type"] not in ["sz", "sd_wave"]:  # reached first non-sz and non-sd segment
        end_frame_i_sz = row["frame_begin"] - 1
        break
    else:
        print(row["interval_type"])
print(begin_frame_i_sz)
print(end_frame_i_sz)

In [ ]:
begin_frame = begin_frame_i_sz - 30  # add 2 sec before
end_frame = end_frame_i_sz + 30  # add 2 sec after

In [ ]:
vid_title = os.path.splitext(os.path.split(fpath_nd2)[-1])[0] + "_onsets"
if "df_onsets_all_sz" in locals():
    if len(df_onsets_all_sz["i_sz"].unique()) > 1:
        vid_title = vid_title + f"_sz{i_sz}"
print(f"Output video root file name: {vid_title}")

In [ ]:
def saveVid(fname, i_begin_frame, i_end_frame):  # in 1-indexing, both endpoints inclusive
    fig = plt.figure(figsize=(18,18))
    canvas_width, canvas_height = fig.canvas.get_width_height()
    gridspec = fig.add_gridspec(1, 1)
    figure=fig.add_subplot(gridspec[0, 0])
    image = figure.imshow(nd2_data[0], cmap="Greens_r")
    figure.axis('off')
    scat = figure.scatter(centers_x, centers_y, color="red", s=0)
    fig.tight_layout()

    
    
    def updateWithSD(frame):
        # update video frame
        image.set_data(nd2_data[frame])
        # show neurons with onset in present or past frames. Do so by setting 
        sizes = np.ones(len(centers_x))*60.0  # create array with circle sizes.
        sizes[onsets_szs_np > frame] = 0.0  # set to invisible all the points that have not fired yet
        colors_arr = np.array(["blue" for i in range(len(centers_x))], dtype='<U6')  # entries should be able to contain yellow, red, white 
        colors_arr[onsets_sd1_np <= frame] = "red"
        colors_arr[onsets_sd2_np <= frame] = "black"
        
        
        # https://stackoverflow.com/questions/9401658/how-to-animate-a-scatter-plot
        scat.set_sizes(sizes)
        scat.set_color(colors_arr)
        # plt.text(50, 50, f"{np.sum(onsets_szs_np > frame)}", size=40, color="red", bbox=dict(fill=False, edgecolor='red', linewidth=2))
    def updateWithSingleSD(frame):
        # update video frame
        image.set_data(nd2_data[frame])
        # show neurons with onset in present or past frames. Do so by setting 
        sizes = np.ones(len(centers_x))*60.0  # create array with circle sizes.
        sizes[onsets_szs_np > frame] = 0.0  # set to invisible all the points that have not fired yet
        colors_arr = np.array(["blue" for i in range(len(centers_x))], dtype='<U6')  # entries should be able to contain yellow, red, white 
        colors_arr[onsets_sd1_np <= frame] = "red"
        
        
        # https://stackoverflow.com/questions/9401658/how-to-animate-a-scatter-plot
        scat.set_sizes(sizes)
        scat.set_color(colors_arr)
        # plt.text(50, 50, f"{np.sum(onsets_szs_np > frame)}", size=40, color="red", bbox=dict(fill=False, edgecolor='red', linewidth=2))
    
    
    def updateNoSD(frame):
        # update video frame
        image.set_data(nd2_data[frame])
        # show neurons with onset in present or past frames. Do so by setting 
        sizes = np.ones(len(centers_x))*60.0  # create array with circle sizes.
        sizes[onsets_szs_np > frame] = 0.0  # set to invisible all the points that have not fired yet
        # https://stackoverflow.com/questions/9401658/how-to-animate-a-scatter-plot
        scat.set_sizes(sizes)
        # plt.text(50, 50, f"{np.sum(onsets_szs_np > frame)}", size=40, color="red", bbox=dict(fill=False, edgecolor='red', linewidth=2))

        
    # Open an ffmpeg process
    outf = os.path.join(fh.open_dir("Choose output folder"), fh.get_filename_with_date(fname, ".mp4"))
    print(outf)
    # lossless encoding:
    # https://stackoverflow.com/questions/37344997/how-to-get-a-lossless-encoding-with-ffmpeg-libx265
    cmdstring = ('ffmpeg', 
                 '-y', '-r', '15', # overwrite, 1fps
                 '-s', '%dx%d' % (canvas_width, canvas_height), # size of image string
                 '-pix_fmt', 'argb', # format
                 '-f', 'rawvideo',  '-i', '-', # tell ffmpeg to expect raw video from the pipe
                 '-vcodec', 'mpeg4', outf) # use mpeg4 encoding
                 #'-c:v', 'libx265',
                 #'-x265-params', '"profile=monochrome12:crf=0:lossless=1:preset=veryslow:qp=0"',
                 #outf)
    p = subprocess.Popen(cmdstring, stdin=subprocess.PIPE, shell=True)
    
    if "onsets_sd2_np" in globals():
        update_func = updateWithSD
        print("Assuming 2 SD waves")
    elif "onsets_sd1_np" in globals():
        update_func = updateWithSingleSD
        print("Assuming 1 SD wave")
    else: 
        update_func = updateNoSD
        print("Assuming no SD waves present")

    # Draw frames and write to the pipe
    for frame in range(i_begin_frame-1, i_end_frame):
        print(frame)
        # draw the frame
        update_func(frame)
        fig.canvas.draw()

        # extract the image as an ARGB string
        string = fig.canvas.tostring_argb()
        # write to pipe
        p.stdin.write(string)

    # Finish up
    p.communicate()

In [ ]:
saveVid(vid_title, begin_frame, end_frame)

In [ ]:
min(onsets_szs_np)

In [ ]:
onsets_szs_np2 = np.array([i for i in range(5471, 5471+715)])

In [ ]:
len(onsets_szs_np2)

In [ ]:
onsets_szs_np > 5466

In [ ]:
fig = plt.figure(figsize=(18,12))
plt.hist(onsets_szs_np, bins=30)
plt.text(5470, 80, "a", size=20, bbox=dict(fill=False, edgecolor='red', linewidth=2))
plt.show()

In [ ]:
# TODO: for some reason, setting the size like it is now does not make the scatterplot on time... Compare with histo